# Projeto 1 - FGV QUANT
## Otimização de Portfólio (Markowitz) e Tomada de Decisão

- A escolha será baseada no Índice Brasil 50 (IBrX 50 B3) data: 20/05/21

- Comparar EW x CW x GMV x MSR \
  . EW = EqualWeighted \
  . CW =  CapWeighted \
  . GMV = Global Minimum Variance \
  . MSR = Maximum Sharpe Ratio

- Parâmetros da Otimização \
  . Risk free = SELIC 10 anos \
  . Apenas Long
 
- Fazer uma Análise de Fatores de Risco (CAPM e o Fama-French)

- Backtest
- Colocar aportes mensais
- Não mudar os pesos

Referências:
https://github.com/HugoAzv/Investment-Management/blob/main/2.%20Advanced%20Portfolio%20Construction%20and%20Analysis%20with%20Python/Module%201%20Lab%20Session%20-%20Foundations.ipynb

https://github.com/HugoAzv/Investment-Management/blob/main/2.%20Advanced%20Portfolio%20Construction%20and%20Analysis%20with%20Python/Module%202%20Lab%20Session%20-%20Covariance%20Estimation.ipynb

In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data
from selenium import webdriver
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [4]:
# SELECIONAR A CARTEIRA, IBXL OU IBOV
indices = ['IBXL', 'IBOV']
url = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{}?language=pt-br'.format(indices[0])

# ABRIR UM DRIVER DO GOOGLE PARA FAZER O DOWNLOAD DO ARQUIVO CSV
'''
como melhorar isso para fazer webscrapping??
tentar pegar depois do Status Invest
https://statusinvest.com.br/indices/indice-brasil-50
'''

driver = webdriver.Chrome('C:\webdrivers\chromedriver.exe')
driver.get(url)
driver.find_element_by_link_text("Download").click()

In [7]:
#### MUDAR O DIA QUANDO FOR PEGAR OS DADOS NOVAMENTE
arquivo_path = 'E:/Users/Hugo/Downloads/IBXLDia_20-05-21.csv'
df = pd.read_csv(arquivo_path, sep=';', skipfooter=2, engine='python', thousands='.', decimal=',', header=1, index_col=False)
df[:8]

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,ABEV3,AMBEV S/A,ON,4358542894,3.471
1,AZUL4,AZUL,PN N2,327262616,0.591
2,BTOW3,B2W DIGITAL,ON NM,201802882,0.518
3,B3SA3,B3,ON EB NM,6119434194,4.779
4,BBSE3,BBSEGURIDADE,ON NM,671584841,0.711
5,BRML3,BR MALLS PAR,ON NM,843728684,0.408
6,BBDC4,BRADESCO,PN N1,4261649634,4.942
7,BBAS3,BRASIL,ON NM,1283197221,1.903


In [40]:
tickers = list(df['Código'][:8])
tickers.append('^BVSP')
tickers

['ABEV3',
 'AZUL4',
 'BTOW3',
 'B3SA3',
 'BBSE3',
 'BRML3',
 'BBDC4',
 'BBAS3',
 '^BVSP']

In [ ]:
# bancos = ['BBAS3.SA', 'BBDC4.SA', 'ITUB4.SA', 'SANB11.SA']
# seguradoras = ['BBSE3.SA', 'SULA11.SA']
# elétrico = ['EGIE3.SA', 'TAEE11.SA', 'TRPL4.SA']
# comunicação = ['VIVT3.SA']
# financeiro = ['B3SA3.SA']
# bovespa = ['^BVSP']
# tickers = bancos + seguradoras + elétrico + comunicação + financeiro + bovespa
# tickers

In [17]:
inicio = '04-11-2017'
fim = 'today'

In [44]:
''' para comentar em bloco: ctrl + / (do teclado numérico)'''
# DataFrame para carregar os preços dos tickers
prices = pd.DataFrame()

for ticker in tickers:
    if ticker != '^BVSP':
        prices[ticker] = data.DataReader(ticker+'.SA', data_source='yahoo', start=inicio, end=fim)['Adj Close']
    else:
        prices[ticker] = data.DataReader(ticker, data_source='yahoo', start=inicio, end=fim)['Adj Close']

# Ajustar o Índice e a coluna do IBOV
prices.index.name = 'Date'
prices = prices.rename(columns = {'^BVSP':'IBOV'})

# Salvar o DataFrame para futuras aplicações
prices.to_csv('portfolio.csv')

portfolio = pd.read_csv('portfolio.csv', header=0, index_col=0, na_values=-99.99)
prices = portfolio.iloc[:,:-1]
prices

,ABEV3,AZUL4,BTOW3,B3SA3,BBSE3,BRML3,BBDC4,BBAS3
Date,,,,,,,,
2017-04-11,15.742681,22.400000,11.663282,5.621583,20.643072,11.320664,17.085791,27.052013
2017-04-12,15.787351,22.959999,11.742356,5.598086,20.434559,11.256796,16.915510,26.606913
2017-04-13,15.867765,22.990000,11.870849,5.436547,20.190088,11.256796,16.399368,25.222176
2017-04-17,16.135799,22.850000,12.147605,5.742002,20.916304,11.544205,17.133671,26.392599
2017-04-18,16.225145,23.910000,12.799958,5.786059,20.844400,11.592106,17.032578,26.648129
...,...,...,...,...,...,...,...,...
2021-05-14,17.180000,39.669998,59.310001,17.850000,23.350000,10.650000,24.889999,31.160000
2021-05-17,17.400000,39.439999,58.500000,17.580000,23.500000,10.910000,25.000000,31.709999
2021-05-18,17.420000,39.580002,57.740002,17.299999,23.270000,10.750000,25.400000,32.150002


In [45]:
ibov = portfolio[['IBOV']]
ibov

,IBOV
Date,
2017-04-11,64360.000000
2017-04-12,63892.000000
2017-04-13,62826.000000
2017-04-17,64335.000000
2017-04-18,64159.000000
...,...
2021-05-14,121881.000000
2021-05-17,122938.000000
2021-05-18,122980.000000


## Pergunta 1: Como escolhi a data de início?

A data de início foi em 11 de Abril de 2017, antes dessa data não existia valores de fechamento da AZUL4.SA

In [46]:
rets = prices.pct_change().dropna()
rets

,ABEV3,AZUL4,BTOW3,B3SA3,BBSE3,BRML3,BBDC4,BBAS3
Date,,,,,,,,
2017-04-12,0.002838,0.025000,0.006780,-0.004180,-0.010101,-0.005642,-0.009966,-0.016454
2017-04-13,0.005094,0.001307,0.010943,-0.028856,-0.011964,0.000000,-0.030513,-0.052044
2017-04-17,0.016892,-0.006090,0.023314,0.056185,0.035969,0.025532,0.044776,0.046405
2017-04-18,0.005537,0.046389,0.053702,0.007673,-0.003438,0.004149,-0.005900,0.009682
2017-04-19,-0.003304,0.000418,0.028571,-0.019797,-0.018627,-0.035813,-0.009997,-0.036499
...,...,...,...,...,...,...,...,...
2021-05-14,0.017773,0.017440,-0.011665,0.034783,-0.000856,0.030978,0.009327,0.000000
2021-05-17,0.012806,-0.005798,-0.013657,-0.015126,0.006424,0.024413,0.004419,0.017651
2021-05-18,0.001149,0.003550,-0.012991,-0.015927,-0.009787,-0.014665,0.016000,0.013876


In [47]:
rets.describe()

,ABEV3,AZUL4,BTOW3,B3SA3,BBSE3,BRML3,BBDC4,BBAS3
count,1017.000000,1017.000000,1017.000000,1017.000000,1017.000000,1017.000000,1017.000000,1017.000000
mean,0.000299,0.001467,0.002282,0.001410,0.000297,0.000362,0.000701,0.000583
std,0.019431,0.041281,0.038278,0.024877,0.018386,0.027567,0.024367,0.028034
min,-0.157818,-0.368677,-0.169724,-0.161203,-0.102251,-0.238820,-0.142744,-0.199054
25%,-0.008716,-0.014182,-0.019231,-0.011129,-0.009548,-0.012978,-0.011800,-0.012760
50%,0.000000,0.001667,-0.000208,0.000448,0.000414,-0.000738,0.000000,0.000405
75%,0.009091,0.017349,0.023314,0.014351,0.009815,0.013371,0.012548,0.013680
max,0.098713,0.298737,0.273954,0.192447,0.107558,0.153614,0.168669,0.171261


In [ ]:
fig = plt.figure()
ax1 = prices.plot(figsize=(20,8), title='Preço de Fechamento Ajustado de Todas as Ações')
ax2 = ax1.twinx()
ibov.plot(ax=ax2, color='black', style='--')
plt.legend()
plt.grid()

In [ ]:
rets.plot(figsize=(20,8), title='Retorno de Todas as Ações')
plt.legend()
plt.grid()

In [ ]:
corr_rets_ibov = rets.join(ibov)
corr_rets_ibov

In [ ]:
plt.figure(figsize = (20,12))
ax = sns.heatmap(corr_rets_ibov.corr(), annot = True)

In [ ]:
plt.figure(figsize = (20,12))
ax = sns.pairplot(corr_rets_ibov)

## Retorno do Ibovespa no período

In [ ]:
# ibov = data.DataReader('^BVSP', data_source='yahoo', start=inicio, end=fim)['Adj Close']
ibov_rets = ibov.pct_change().dropna()
ibov_rets

In [ ]:
plt.figure(figsize = (20,12))
ax = sns.distplot(ibov_rets)

In [ ]:
rets_sum = (rets + 1).cumprod()
rets_sum_ibov = (ibov_rets+1).cumprod()
rets_sum

In [ ]:
fig = plt.figure()
ax1 = rets_sum.plot(title='Retorno x Tempo', figsize = (20,12))

ax2 = ax1.twinx()
rets_sum_ibov.plot(ax=ax2, color='black', style='--')